<a href="https://colab.research.google.com/github/lakshitgosain/Finetuning-LLM/blob/main/LLM_FinetuningLLM_P2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
import itertools
import jsonlines

from datasets import load_dataset
from pprint import pprint

import llama
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

In [12]:
!pip install llama-llm

In [20]:
instruction_tuned_dataset=load_dataset("tatsu-lab/alpaca", split="train", streaming=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [34]:
m=5
print("Instruction-tuned Dataset:")
top_m=list(itertools.islice(instruction_tuned_dataset,m))
for j in top_m:
  print(j)

Instruction-tuned Dataset:
{'instruction': 'Give three tips for staying healthy.', 'input': '', 'output': '1.Eat a balanced diet and make sure to include plenty of fruits and vegetables. \n2. Exercise regularly to keep your body active and strong. \n3. Get enough sleep and maintain a consistent sleep schedule.', 'text': 'Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\nGive three tips for staying healthy.\n\n### Response:\n1.Eat a balanced diet and make sure to include plenty of fruits and vegetables. \n2. Exercise regularly to keep your body active and strong. \n3. Get enough sleep and maintain a consistent sleep schedule.'}
{'instruction': 'What are the three primary colors?', 'input': '', 'output': 'The three primary colors are red, blue, and yellow.', 'text': 'Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\nWhat are the three p

In [35]:
prompt_template_with_input="""Below is an instruction on that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{instruction}

### Input:
{input}

### Response:"""

prompt_template_without_input="""Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
{instruction}

### Response:"""



In [36]:
processed_data=[]
for j in top_m:
  if not j['input']:
    processed_prompt=prompt_template_without_input.format(instruction=j['instruction'])
  else:
    processed_prompt = prompt_template_with_input.format(instruction=j["instruction"], input=j["input"])

  processed_data.append({"input": processed_prompt, "output": j['output']})

In [38]:
pprint(processed_data[0])

{'input': 'Below is an instruction that describes a task. Write a response '
          'that appropriately completes the request.\n'
          '\n'
          '### Instruction:\n'
          'Give three tips for staying healthy.\n'
          '\n'
          '### Response:',
 'output': '1.Eat a balanced diet and make sure to include plenty of fruits '
           'and vegetables. \n'
           '2. Exercise regularly to keep your body active and strong. \n'
           '3. Get enough sleep and maintain a consistent sleep schedule.'}


In [39]:
with jsonlines.open(f"alpaca_processed.json",'w') as writer:
  writer.write_all(processed_data)

In [40]:
#Run non instruction tuned model
#Run instruction tuned model

###Data Preparation

In [44]:
import pandas as pd
import datasets
from pprint import pprint
from transformers import AutoTokenizer, AutoModelForCausalLM

In [45]:
tokenizer=AutoTokenizer.from_pretrained("EleutherAI/pythia-70m")

tokenizer_config.json:   0%|          | 0.00/396 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

In [47]:
text="Hi How are you?"
encoded_text=tokenizer(text)["input_ids"]

In [48]:
encoded_text

[12764, 1359, 403, 368, 32]

In [49]:
decoded_text=tokenizer.decode(encoded_text)
print("Decoded Tokens back into text are:",decoded_text)

Decoded Tokens back iinto ext are Hi How are you?


In [50]:
list_texts = ["Hi, how are you?", "I'm good", "Yes"]
encoded_texts = tokenizer(list_texts)
print("Encoded several texts: ", encoded_texts["input_ids"])

Encoded several texts:  [[12764, 13, 849, 403, 368, 32], [42, 1353, 1175], [4374]]


In [51]:
tokenizer.pad_token=tokenizer.eos_token
encode_texts_longest=tokenizer(list_texts,padding=True)
print("Using Padding:",encode_texts_longest['input_ids'])

Using Padding: [[12764, 13, 849, 403, 368, 32], [42, 1353, 1175, 0, 0, 0], [4374, 0, 0, 0, 0, 0]]


In [53]:
encoded_text_truncation=tokenizer(list_texts,max_length=3, truncation=True)
print("using Truncation:", encoded_text_truncation['input_ids'])

using Truncation: [[12764, 13, 849], [42, 1353, 1175], [4374]]


In [59]:
filename="lamini_docs.jsonl"

instruction_dataset_df=pd.read_json(filename,lines=True)
examples=instruction_dataset_df.to_dict()

if "question" in examples and "answer" in examples:
  text=examples['question'][0]+examples['answer'][0]
elif "intruction" in examples and "response" in examples:
  text=examples['instruction'][0] + examples['response'][0]
elif "input" in examples and "output" in examples:
  text=examples['input'][0] + examples['output'][0]
else:
  text=examples['text'][0]

prompt_template = """### Question:
{question}

### Answer:"""

num_examples=len(examples['question'])
finetuning_dataset=[]
for i in range(num_examples):
  question= examples["question"][1]
  answer= examples['answer'][1]
  text_with_prompt_template=prompt_template.format(question=question)
  finetuning_dataset.append({"question":text_with_prompt_template,"answer":answer})

print("One datapoint in the finetuning dataset:")
pprint(finetuning_dataset[0])

One datapoint in the finetuning dataset:
{'answer': 'Lamini can be downloaded as a python package and used in any '
           'codebase that uses python. Additionally, we provide a language '
           'agnostic REST API. We’ve seen users develop and train models in a '
           'notebook environment, and then switch over to a REST API to '
           'integrate with their production environment.',
 'question': '### Question:\n'
             'What is the recommended way to set up and configure the code '
             'repository?\n'
             '\n'
             '### Answer:'}


In [61]:
text=finetuning_dataset[0]['question'] + finetuning_dataset[0]['answer']
tokenized_inputs=tokenizer(
    text,
    return_tensors='np',
    padding=True
)

In [63]:
tokenized_inputs['input_ids']

array([[ 4118, 19782,    27,   187,  1276,   310,   253,  8521,  1039,
          281,   873,   598,   285, 20486,   253,  2127, 18491,    32,
          187,   187,  4118, 37741,    27,    45,  4988,    74,   476,
          320, 20582,   347,   247, 15548,  5522,   285,   908,   275,
          667,  2127,  4793,   326,  4648, 15548,    15,  9157,    13,
          359,  2085,   247,  3448,   639,    79,  6932, 30392,  8990,
           15,   844,   457,   306,  2326,  4212,  1287,   285,  6194,
         3210,   275,   247, 24849,  3126,    13,   285,   840,  5234,
          689,   281,   247, 30392,  8990,   281, 19837,   342,   616,
         3275,  3126,    15]])

In [66]:
max_length=2048
max_length=min(
    tokenized_inputs["input_ids"].shape[1],
    max_length
)

In [67]:
text=finetuning_dataset[0]['question'] + finetuning_dataset[0]['answer']
tokenized_inputs=tokenizer(
    text,
    return_tensors='np',
    truncation=True,
    max_length=max_length
)

In [68]:
tokenized_inputs['input_ids']

array([[ 4118, 19782,    27,   187,  1276,   310,   253,  8521,  1039,
          281,   873,   598,   285, 20486,   253,  2127, 18491,    32,
          187,   187,  4118, 37741,    27,    45,  4988,    74,   476,
          320, 20582,   347,   247, 15548,  5522,   285,   908,   275,
          667,  2127,  4793,   326,  4648, 15548,    15,  9157,    13,
          359,  2085,   247,  3448,   639,    79,  6932, 30392,  8990,
           15,   844,   457,   306,  2326,  4212,  1287,   285,  6194,
         3210,   275,   247, 24849,  3126,    13,   285,   840,  5234,
          689,   281,   247, 30392,  8990,   281, 19837,   342,   616,
         3275,  3126,    15]])

In [69]:
def tokenize_function(examples):
    if "question" in examples and "answer" in examples:
      text = examples["question"][0] + examples["answer"][0]
    elif "input" in examples and "output" in examples:
      text = examples["input"][0] + examples["output"][0]
    else:
      text = examples["text"][0]

    tokenizer.pad_token = tokenizer.eos_token
    tokenized_inputs = tokenizer(
        text,
        return_tensors="np",
        padding=True,
    )

    max_length = min(
        tokenized_inputs["input_ids"].shape[1],
        2048
    )
    tokenizer.truncation_side = "left"
    tokenized_inputs = tokenizer(
        text,
        return_tensors="np",
        truncation=True,
        max_length=max_length
    )

    return tokenized_inputs

In [70]:
finetuning_dataset_loaded = datasets.load_dataset("json", data_files=filename, split="train")

tokenized_dataset = finetuning_dataset_loaded.map(
    tokenize_function,
    batched=True,
    batch_size=1,
    drop_last_batch=True
)

print(tokenized_dataset)

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/1400 [00:00<?, ? examples/s]

Dataset({
    features: ['question', 'answer', 'input_ids', 'attention_mask'],
    num_rows: 1400
})


In [71]:
tokenized_dataset = tokenized_dataset.add_column("labels", tokenized_dataset["input_ids"])

In [72]:
split_dataset = tokenized_dataset.train_test_split(test_size=0.1, shuffle=True, seed=123)
print(split_dataset)

DatasetDict({
    train: Dataset({
        features: ['question', 'answer', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 1260
    })
    test: Dataset({
        features: ['question', 'answer', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 140
    })
})
